In [2]:
pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 33.9 MB/s eta 0:00:00


In [3]:
#Cleaned + Ensemble (Distilbert + Bertweet)

import pandas as pd
import numpy as np
import torch
import re
import random
import emoji
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Set seeds for full reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Load dataset
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Preprocessing function
def preprocess_text(text):
    if pd.isna(text):
        return ""
    # Replace emojis with textual descriptions
    text = emoji.replace_emoji(text, replace=lambda chars, data_dict: ' '.join(data_dict['en'].split('_')).strip(':'))
    # Remove URLs
    text = re.sub(r"http\S+|www\S+", "", text)
    # Remove mentions
    text = re.sub(r"@\w+", "", text)
    # Remove hashtags but keep the word (e.g., #earthquake -> earthquake)
    text = re.sub(r"#", "", text)
    # Remove special characters and extra spaces
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    return text.lower().strip()

# Apply preprocessing to the dataset
for df in [train_df, test_df]:
    df['text'] = df['text'].fillna('').apply(preprocess_text)
    df['keyword'] = df['keyword'].fillna('').str.replace('%20', ' ').apply(preprocess_text)
    df['combined'] = df['keyword'] + " " + df['text']

# Feature Engineering: Add sentiment polarity and word count
def extract_sentiment(text):
    return TextBlob(text).sentiment.polarity

for df in [train_df, test_df]:
    df['sentiment'] = df['text'].apply(extract_sentiment)
    df['word_count'] = df['text'].apply(lambda x: len(x.split()))
    df['exclamation_count'] = df['text'].str.count('!')

# Save cleaned data
train_df.to_csv("cleaned_train.csv", index=False)
test_df.to_csv("cleaned_test.csv", index=False)

print("Data cleaning complete!")

# Tokenizers
tokenizer_distilbert = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer_bertweet = AutoTokenizer.from_pretrained("vinai/bertweet-base")

# Tokenization function
def encode_text(texts, tokenizer, max_length=256):
    return tokenizer(texts.tolist(), padding=True, truncation=True, max_length=max_length, return_tensors='pt')

# Encode text data
X_distilbert = encode_text(train_df['combined'], tokenizer_distilbert)
X_bertweet = encode_text(train_df['combined'], tokenizer_bertweet)
y = torch.tensor(train_df['target'].values)

# Split dataset
X_train_d, X_val_d, y_train, y_val = train_test_split(X_distilbert['input_ids'], y, test_size=0.1, random_state=SEED)
X_train_b, X_val_b, _, _ = train_test_split(X_bertweet['input_ids'], y, test_size=0.1, random_state=SEED)

X_train_mask_d, X_val_mask_d = train_test_split(X_distilbert['attention_mask'], test_size=0.1, random_state=SEED)
X_train_mask_b, X_val_mask_b = train_test_split(X_bertweet['attention_mask'], test_size=0.1, random_state=SEED)

# Use a seeded generator for DataLoader reproducibility
generator = torch.Generator().manual_seed(SEED)

# Create DataLoaders
batch_size = 32
train_loader_d = DataLoader(TensorDataset(X_train_d, X_train_mask_d, y_train), batch_size=batch_size, shuffle=True, generator=generator)
val_loader_d = DataLoader(TensorDataset(X_val_d, X_val_mask_d, y_val), batch_size=batch_size, generator=generator)

train_loader_b = DataLoader(TensorDataset(X_train_b, X_train_mask_b, y_train), batch_size=batch_size, shuffle=True, generator=generator)
val_loader_b = DataLoader(TensorDataset(X_val_b, X_val_mask_b, y_val), batch_size=batch_size, generator=generator)

# Initialize models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_distilbert = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to(device)
model_bertweet = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=2).to(device)

optimizer_d = AdamW(model_distilbert.parameters(), lr=1e-5, eps=1e-8)
optimizer_b = AdamW(model_bertweet.parameters(), lr=2e-5, eps=1e-8)

# Training function
def train_model(model, train_loader, val_loader, optimizer, model_name):
    best_acc = 0
    patience = 3
    no_improve_epochs = 0
    epochs = 5

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc=f"Training {model_name} (Epoch {epoch + 1})"):
            optimizer.zero_grad()
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch + 1} Loss ({model_name}): {total_loss / len(train_loader):.4f}")

        # Validation
        model.eval()
        val_preds, val_true = [], []

        with torch.no_grad():
            for batch in val_loader:
                input_ids, attention_mask, labels = [b.to(device) for b in batch]
                outputs = model(input_ids, attention_mask=attention_mask)
                preds = torch.argmax(outputs.logits, dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_true.extend(labels.cpu().numpy())

        acc = accuracy_score(val_true, val_preds)
        print(f"{model_name} - Validation Accuracy: {acc:.4f}")

        # Early stopping
        if acc > best_acc:
            best_acc = acc
            no_improve_epochs = 0
            torch.save(model.state_dict(), f"{model_name}_best.pth")
        else:
            no_improve_epochs += 1
            if no_improve_epochs >= patience:
                print(f"Early stopping for {model_name}!")
                break

# Train both models
train_model(model_distilbert, train_loader_d, val_loader_d, optimizer_d, "distilbert")
train_model(model_bertweet, train_loader_b, val_loader_b, optimizer_b, "bertweet")

# Load best models
model_distilbert.load_state_dict(torch.load("distilbert_best.pth"))
model_bertweet.load_state_dict(torch.load("bertweet_best.pth"))

# Prepare test data
X_test_d = encode_text(test_df['combined'], tokenizer_distilbert)
X_test_b = encode_text(test_df['combined'], tokenizer_bertweet)

test_loader_d = DataLoader(TensorDataset(X_test_d['input_ids'], X_test_d['attention_mask']), batch_size=batch_size, generator=generator)
test_loader_b = DataLoader(TensorDataset(X_test_b['input_ids'], X_test_b['attention_mask']), batch_size=batch_size, generator=generator)

# Function to get predictions
def get_predictions(model, test_loader):
    model.eval()
    preds = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask = [b.to(device) for b in batch]
            outputs = model(input_ids, attention_mask=attention_mask)
            preds.extend(torch.softmax(outputs.logits, dim=1)[:, 1].cpu().numpy())
    return np.array(preds)

# Get predictions
preds_d = get_predictions(model_distilbert, test_loader_d)
preds_b = get_predictions(model_bertweet, test_loader_b)

# Ensemble and save
final_preds = ((preds_d + preds_b) / 2 >= 0.5).astype(int)
submission_df = pd.read_csv("sample_submission.csv")
submission_df["target"] = final_preds
submission_df.to_csv("ensemble_submission.csv", index=False)
print("Predictions saved to ensemble_submission.csv")


Data cleaning complete!


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Epoch 4 Loss (bertweet): 0.2350
bertweet - Validation Accuracy: 0.8346


Training bertweet (Epoch 5): 100%|██████████| 215/215 [00:25<00:00,  8.35it/s]


Epoch 5 Loss (bertweet): 0.1967
bertweet - Validation Accuracy: 0.8228


<ipython-input-3-28cca9ef5df8>:161: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_distilbert.load_state_dict(torch.load("distilbert_best.pth"))
<ipython-input-3-28cca9

Predictions saved to ensemble_submission.csv
